### Import libraries

In [60]:
import cv2
import csv

### Define paramerters and API key

In [61]:
# Your PAT (Personal Access Token) can be found in the portal under Authentification
PAT = '858d33cd787c47ccb0ab96e6728dbfd4'
# Specify the correct user_id/app_id pairings
# Since you're making inferences outside your app's scope
USER_ID = 'leumas'
APP_ID = 'my-first-application'
# Change these to whatever model and image URL you want to use
MODEL_ID = 'ocr-scene-english-paddleocr'
MODEL_VERSION_ID = '40dbb2c9cde44a27af226782e7157006'
IMAGE_FILE_LOCATION = './image/input_half.png'

############################################################################
# YOU DO NOT NEED TO CHANGE ANYTHING BELOW THIS LINE TO RUN THIS EXAMPLE
############################################################################

from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)

metadata = (('authorization', 'Key ' + PAT),)

userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)

with open(IMAGE_FILE_LOCATION, "rb") as f:
    file_bytes = f.read()

post_model_outputs_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        user_app_id=userDataObject,  # The userDataObject is created in the overview and is required when using a PAT
        model_id=MODEL_ID,
        version_id=MODEL_VERSION_ID,  # This is optional. Defaults to the latest model version
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        base64=file_bytes
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
    print(post_model_outputs_response.status)
    raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)

# Since we have one input, one output will exist here
output = post_model_outputs_response.outputs[0]

print("Predicted concepts:")
for concept in output.data.concepts:
    print("%s %.2f" % (concept.name, concept.value))

# Uncomment this line to print the full Response JSON
#print(output)

Predicted concepts:


### Importing original image to refer original size

In [62]:
image = cv2.imread(IMAGE_FILE_LOCATION)
image.shape

(2524, 2852, 3)

### Define function to get bounding box coordinates and write to solo file

In [63]:
with open('ocr_bb.csv', 'w') as myfile:
    for box in post_model_outputs_response.outputs[0].data.regions:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        name = box.data.text.raw
        top = int(box.region_info.bounding_box.top_row * image.shape[0])
        left = int(box.region_info.bounding_box.left_col * image.shape[1])
        bottom = int(box.region_info.bounding_box.bottom_row * image.shape[0])
        right = int(box.region_info.bounding_box.right_col * image.shape[1])

        wr.writerow([name, top, left, bottom, right])

### Define function to get bounding box coordinates and write to common.csv file

In [64]:
with open('common.csv', 'a') as myfile:
    for box in post_model_outputs_response.outputs[0].data.regions:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        name = box.data.text.raw
        top = int(box.region_info.bounding_box.top_row * image.shape[0])
        left = int(box.region_info.bounding_box.left_col * image.shape[1])
        bottom = int(box.region_info.bounding_box.bottom_row * image.shape[0])
        right = int(box.region_info.bounding_box.right_col * image.shape[1])

        wr.writerow(["Dimension Text", name, -1, -1, left, top, right, bottom, -1])